# predict on sequence

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)

from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')

In [ ]:
import notebook_helper as helper
predictpath="data/predictions/"

'''
# ref/ref
helper.run(rs_model=False,rs_structure=False, filename=predictpath+"shaker_ref_ref.react", maxstruct_train=3, maxstruct_predict=3)
# ref/rs
helper.run(rs_model=False,rs_structure=True, filename=predictpath+"shaker_ref_rs_1str.react", maxstruct_train=3, maxstruct_predict=1)
helper.run(rs_model=False,rs_structure=True, filename=predictpath+"shaker_ref_rs_3str.react", maxstruct_train=3, maxstruct_predict=3)
# ref/rs
helper.run(rs_model=False,rs_structure=True, filename=predictpath+"shaker_ref_rs_5str.react", maxstruct_train=3, maxstruct_predict=5)
# ref/rs
helper.run(rs_model=False,rs_structure=True, filename=predictpath+"shaker_ref_rs_7str.react", maxstruct_train=3, maxstruct_predict=7)
# rs/rs
helper.run(rs_model=True,rs_structure=True, filename=predictpath+"shaker_rs_rs_33.react", maxstruct_train=3, maxstruct_predict=3)
# rs/rs
helper.run(rs_model=True,rs_structure=True, filename=predictpath+"shaker_rs_rs_55.react", maxstruct_train=5, maxstruct_predict=5)
helper.run(rs_model=True,rs_structure=True, filename=predictpath+"shaker_rs_rs_77.react", maxstruct_train=7, maxstruct_predict=7)
helper.run(rs_model=True,rs_structure=True, filename=predictpath+"shaker_rs_rs_100100.react", maxstruct_train=100, maxstruct_predict=100)
helper.run(rs_model=False,rs_structure=True, filename=predictpath+"shaker_ref_rs_100.react", maxstruct_train=100, maxstruct_predict=100)
'''
shakerdata = ["shaker_ref_ref.react","shaker_ref_rs_1str.react","shaker_ref_rs_3str.react",
              "shaker_ref_rs_5str.react","shaker_ref_rs_7str.react","shaker_ref_rs_100.react",
              "shaker_rs_rs_33.react", "shaker_rs_rs_55.react","shaker_rs_rs_77.react", "shaker_rs_rs_100100.react"]

# DRAW FUNCTION FOR THE SHAPE DATA

In [ ]:
from graphlearn01.utils import draw
import numpy as np
from numpy import around
from scipy.stats import pearsonr as pear
from scipy.stats import spearmanr as spear
import simushape  as ss
from IPython.display import HTML, display
import tabulate
import simushape_nostruct  as sn


def annotate(g,shap):
    n=g.nodes()
    n.sort()
    for e,i in zip(n,shap):
        g.node[e]["importance"]= [i]
        g.node[e]["none"]= ''
    return g

def draw3(graph, shape_list):
    graphs=[annotate(graph.copy(),shape) for shape in shape_list  ]
    draw.graphlearn(graphs,size=15,layout="RNA",vertex_label='none',edge_alpha=0.05,vertex_size=150,vertex_border=False, scoretricks=True, n_graphs_per_line=len(shape_list))
    
    
def table_adder(tables):
    # flip 
    res = []
    tables = list(zip(*tables))
    for table in tables:
        tab =list(zip(*table))
        res.append([tab[0][0]]+ [np.array(e).mean()for e in tab[1:]])
    return res
        
def mycorr(pre,train, prediction):
    train = np.array(train)
    prediction = np.array(prediction)
    predict2, train2 = ss.mask(prediction,train)
    p =  pear(train2,predict2)[0]
    s,pvalue =  spear(train2,predict2)
    v =  predict2.var()
    #return "pearson: %.3f spearman %.3f variances: %.3f %.3f" % (p,s, train2.var(), predict2.var())
    return [pre, around(p,3),around(s,3),pvalue,around( v,3)]


klmer2 = ss.read_react("data/predictions/RNA16_hlmer_prediction.react")
FAKE = ss.read_react("data/predictions/RNA16_fake_shapes.react")
sukust = ss.read_react("data/predictions/RNA16_sukosd_prediction.react")
#mfe = ss.read_react("data/predictions/shaker_prediction_ss_mfe.react")

shaker = [ ss.read_react(predictpath+i) for i in shakerdata ]


RNA16 =  ss.read_react("data/RNA16.react")
allreacts = [RNA16]+shaker+[klmer2,FAKE]

RNA16_complete =  helper.get_test_data()

alltables= []
for e in RNA16_complete:
    print "rna name", e
    graph = ss.eden_rna.sequence_dotbracket_to_graph(RNA16_complete[e][1],RNA16_complete[e][2])
    graph.graph["structure"] = RNA16_complete[e][2]
    draw3(graph,[ react[e] for react in allreacts ])
    
    print RNA16_complete[e][2], "<- given structure"
    for z,energy in zip(*sn.shape(RNA16_complete[e][1]))[:3]:
        print z, energy
        
    table = [['technology','pearson','spearman','spear p','variance']]
    for i, experimentname in enumerate(shakerdata):
        table.append(mycorr(experimentname,RNA16[e],shaker[i][e]))
          
    table.append(mycorr("hlmer:",   RNA16[e],klmer2[e]))
    table.append(mycorr("fakeset:",RNA16[e],FAKE[e]))
    
    alltables.append(table[1:])
    display(HTML(tabulate.tabulate(table, tablefmt='html')))  
    #print "sukosd:",mycorr(RNA16[e],sukust[e])
    
    

display(HTML(tabulate.tabulate(table_adder(alltables), tablefmt='html')))     

In [ ]:

from graphlearn01.utils import draw
import numpy as np
from scipy.stats import pearsonr as pear
from scipy.stats import spearmanr as spear
import simushape  as ss
from graphlearn01.utils import draw
import numpy as np
from numpy import around
from scipy.stats import pearsonr as pear
from scipy.stats import spearmanr as spear
import simushape  as ss
from IPython.display import HTML, display
import tabulate
import simushape_nostruct  as sn
def annotate(g,shap):
    n=g.nodes()
    n.sort()
    for e,i in zip(n,shap):
        g.node[e]["importance"]= [i]
        g.node[e]["none"]= ''
    return g


RNA20 =  ss.read_react("data/RNA20.react")
allreacts = [RNA20]
RNA20_complete = ss.get_all_data('data/RNA20.react','data/RNA20.dbn')# load includes sanity check;; name -> react,sequence,dotbracket


def draw_deflayout(graph, shape_list):
    graphs=[annotate(graph.copy(),shape) for shape in shape_list  ]
    draw.graphlearn(graphs,size=15,layout="RNA",vertex_label='none',edge_alpha=0.05,vertex_size=150,vertex_border=False, scoretricks=True)
    

for e in RNA20_complete:
    print "rna name", e 
    print 'length %d' % len(RNA20_complete[e][1])
    graph = ss.eden_rna.sequence_dotbracket_to_graph(RNA20_complete[e][1],RNA20_complete[e][2])
    graph.graph["structure"] = RNA20_complete[e][2]
    draw_deflayout(graph,[ RNA20[e] ])
  
  

In [ ]:
import simushape_nostruct  as sn
import simushape as ss
import numpy as np
# GENERATE FAKE PREDICTIONS

if True:
    data = ss.get_all_data('data/RNA16.react','data/RNA16.dbn')
    #data2 = ss.get_all_data('data/RNA20.react','data/RNA20.dbn')
    #data.pop("GLYCFN",None) # data is bad
    data.pop("23sRNA",None) #2 long for RNAshapes
    data.pop("R009",None)   #2 long for shapes
    predict_keys= data.keys()
    #data.update(data2)
    res=[]
    for e in predict_keys: 
        # just 
        shapes = sn.shape(data[e][1])[0]
        print shapes,'shapes i got'
        results = np.array( [[1 if z=="." else 0 for z in line ] for line in shapes ] )
        print results, 'annotation, 0,1'
        print e, 'seq name'
        my_react = results.mean(axis=0)
        print my_react, 'REACT thats written'
        res.append(">%s"%e)
        res.append('\n'.join(["%s\t%.4f" % (i,e) for i,e in enumerate(my_react)]))
        res.append('')
        print 'x',
        
 
    with open("data/predictions/RNA16_fake_shapes.react",'w') as f: f.write('\n'.join(res))
